You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:

!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
!pip install -U openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [5]:
import os
from google.colab import userdata
openai_api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [6]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [7]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [8]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [9]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [10]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [11]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [16]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=0
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [20]:
result = crew.kickoff(inputs={"topic": "Indian Institute of Technology"})



> Entering new CrewAgentExecutor chain...
I now can give a great answer. 

Final Answer: 

### Comprehensive Content Plan for Blog Article on Indian Institute of Technology (IIT)

---

**1. Latest Trends, Key Players, and Noteworthy News:**

- **Trends:**
  - **Technological Advancements:** IITs are at the forefront of emerging technologies such as AI, machine learning, and blockchain.
  - **International Collaborations:** Increasing partnerships with global universities and industries to enhance research and innovation.
  - **Startup Ecosystem:** IITs are becoming incubators for startups; many alumni are launching successful tech companies.
  - **Focus on Sustainability:** IITs are emphasizing research in sustainable engineering and green technologies.

- **Key Players:**
  - **IIT Bombay:** Known for its strong engineering programs and entrepreneurial ecosystem.
  - **IIT Delhi:** Focuses on cutting-edge research and has a robust placement record.
  - **IIT Madras:** Renowned for i

- Display the results of your execution as markdown in the notebook.

In [21]:
from IPython.display import Markdown
Markdown(result)

```markdown
# Exploring the Indian Institute of Technology: A Gateway to Innovation and Excellence

## Introduction
The Indian Institutes of Technology (IITs) stand as a beacon of excellence in India's educational landscape, primarily focusing on engineering and technology. Established in 1951, these institutions have evolved into world-class centers for education, research, and innovation. With their global recognition, as evidenced by recent rankings in the top 200 universities by QS World University Rankings, IITs continue to shape the future of technology in India and beyond. This blog post delves into the latest trends, academic programs, and the overall impact of IITs on society, making it essential reading for prospective students, parents, and industry professionals.

As we explore the significance of IITs, it is important to recognize their role not just as educational institutions, but as catalysts for technological advancement and societal change. They have consistently produced skilled professionals who contribute to various sectors, thereby influencing both local and global landscapes. This multifaceted impact underscores the necessity of understanding what IITs have to offer.

## History and Evolution of IITs
The IITs were conceived as a response to the growing need for skilled engineers and technologists in post-independence India. The first IIT was established in Kharagpur, and the vision was to create institutions that would not only focus on education but also on research and innovation. Over the decades, the IITs have expanded to include 23 institutes across the country, each contributing uniquely to India's technological landscape.

The impact of IITs on India's development is profound; they have trained generations of engineers and technologists who have gone on to contribute significantly to various sectors, both nationally and internationally. The rigorous curriculum and emphasis on research have placed IITs at the forefront of technological advancements, making them a critical player in the global education system.

## Academic Programs Offered
IITs offer a diverse range of academic programs, including undergraduate, postgraduate, and doctoral courses, with a focus on engineering and technology. The undergraduate programs, primarily the Bachelor of Technology (B.Tech), are highly competitive and cater to a variety of specializations. Additionally, IITs have introduced specialized courses in emerging fields such as Artificial Intelligence, Machine Learning, and Blockchain Technology, ensuring that students are well-prepared for the demands of the modern workforce.

Graduate programs at IITs are equally impressive, with opportunities for research in cutting-edge areas, allowing students to collaborate with industry leaders and contribute to significant projects. The robust academic framework, combined with experienced faculty, ensures that students receive a comprehensive education that equips them for future challenges. This commitment to academic excellence fosters a generation of innovators ready to meet the evolving needs of society.

## Research and Innovation
Research at IITs is a cornerstone of their academic structure. Each IIT engages in various research initiatives that address societal challenges, ranging from sustainable engineering to healthcare innovations. Notable projects have led to patents and technologies that have had a meaningful impact on both local and global scales.

Furthermore, IITs are increasingly collaborating with international universities and industries, fostering a culture of innovation. These partnerships enhance research capabilities and provide students with exposure to global challenges and solutions, further solidifying IITs' role as leaders in technological advancement. Through these initiatives, IITs are not only contributing to academic knowledge but also driving real-world applications that benefit society.

## Campus Life and Facilities
Life at IIT is vibrant, offering students a blend of academics and extracurricular activities. The campuses are equipped with state-of-the-art facilities, including modern classrooms, laboratories, and libraries, ensuring an enriching learning environment. Hostels provide comfortable accommodations, fostering a sense of community among students.

In addition to academics, IITs emphasize the importance of mental health and well-being. Support systems, including counseling services and academic guidance, are in place to help students navigate the pressures of rigorous academic life. This holistic approach contributes to a balanced campus life, allowing students to thrive both personally and academically. By prioritizing student well-being, IITs create an environment conducive to learning and personal growth.

## Placement and Alumni Success Stories
IITs boast impressive placement records, with top companies eager to recruit graduates. The placement cells at each IIT work diligently to connect students with potential employers, ensuring that graduates are well-prepared for their careers. Statistics show that IIT graduates are among the highest earners in the engineering field, with many securing positions in leading global firms.

The success stories of IIT alumni are numerous, with former students making significant contributions in technology, business, and public service. Notable alumni have founded successful startups, led innovative projects, and held key positions in multinational corporations, showcasing the caliber of education and training received at IITs. These achievements not only reflect the quality of education at IITs but also inspire current students to pursue their own entrepreneurial and professional dreams.

## Challenges and Opportunities
Despite their success, IITs face challenges in maintaining the quality of education and research amid growing demand. As more students aspire to join these prestigious institutions, the admission process becomes increasingly competitive. Additionally, the need for continuous funding and resources to support research initiatives remains a pressing issue.

However, these challenges also present opportunities for growth and development. With government initiatives aimed at enhancing research funding and international collaborations, IITs are well-positioned to expand their influence and continue contributing to technological advancements. By addressing these challenges proactively, IITs can ensure their legacy of excellence continues for future generations.

## Conclusion
The Indian Institutes of Technology are not just educational institutions; they are vital players in the landscape of global technology and innovation. Their commitment to excellence in education, research, and societal impact makes them a preferred choice for aspiring engineers and technologists. As we look to the future, IITs will undoubtedly continue to shape the narrative of technological progress in India and around the world.

## Call to Action
For those considering a future at one of the IITs, explore the official IIT websites for the latest updates on admissions and programs. Attending webinars or open days can provide valuable insights into what these esteemed institutions have to offer. Your journey towards innovation and excellence could very well begin at an IIT.
```

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [17]:
topic = "Srila Prabupada - Journey to America"
result = crew.kickoff(inputs={"topic": topic})



> Entering new CrewAgentExecutor chain...
I now can give a great answer.  
Final Answer: 

### Comprehensive Content Plan Document: Srila Prabupada - Journey to America

---

#### 1. Latest Trends, Key Players, and Noteworthy News

**Latest Trends:**
- Increasing interest in Eastern spirituality and its impact on Western culture.
- Growth of online communities centered around Krishna consciousness and ISKCON (International Society for Krishna Consciousness).
- Rising popularity of yoga, meditation, and holistic living, often linked to the teachings of Srila Prabupada.

**Key Players:**
- A.C. Bhaktivedanta Swami Srila Prabupada: Founder of ISKCON, pivotal figure in the dissemination of Gaudiya Vaishnavism in the West.
- ISKCON: The organization continues to thrive globally, with temples, festivals, and educational programs.
- Influencers in spirituality and wellness who reference Srila Prabupada’s teachings in their work.

**Noteworthy News:**
- Recent anniversaries of significant ev

In [19]:
Markdown(result)

```markdown
# Srila Prabupada: His Inspiring Journey to America and the Birth of ISKCON

## Introduction

A.C. Bhaktivedanta Swami Srila Prabupada stands as a monumental figure in the narrative of spirituality in the West. His journey from India to America in 1965 marked a pivotal moment in the dissemination of Krishna consciousness, a branch of Gaudiya Vaishnavism. This exploration reflects on the transformative impact of his journey, which not only shaped his own life but also left an indelible mark on the spiritual landscape of the West, paving the way for the establishment of the International Society for Krishna Consciousness (ISKCON).

The significance of Srila Prabupada's journey is underscored by the increasing interest in Eastern spirituality today. As more individuals seek authentic spiritual experiences amid the challenges of modern materialism, the teachings of Srila Prabupada continue to resonate, inviting a deeper exploration into the rich traditions of India. His enduring legacy is a testament to the universal quest for meaning and connection that transcends cultural boundaries.

## Early Life and Spiritual Awakening

Born in 1896 in Calcutta, India, Srila Prabupada was steeped in the rich cultural and spiritual milieu of his homeland. His early life was profoundly influenced by the teachings of his spiritual mentor, Bhaktisiddhanta Sarasvati Thakura, who emphasized the importance of disseminating Vedic knowledge to the Western world. This nurturing environment ignited in him a profound spiritual curiosity that would later fuel his mission abroad.

Prabupada’s early spiritual journey was marked by intense dedication and practice. He embraced the principles of bhakti yoga, devoting himself to the service of Krishna. This foundational experience not only shaped his personal spirituality but also equipped him with the insights necessary for his future endeavors in the West. His background in traditional Indian philosophy and culture would become instrumental as he navigated the challenges of introducing these concepts to a new audience, bridging the gap between East and West.

## The Decision to Go to America

The decision to journey to America was driven by Prabupada's deep desire to spread Krishna consciousness. He believed that the West, often perceived as materialistic, was in dire need of spiritual enlightenment. His motivations were not without challenges; he faced significant barriers, including language obstacles, cultural differences, and financial constraints.

Despite these hurdles, Srila Prabupada's determination never wavered. He secured a small amount of money, a few treasured books, and a heartfelt mission. It was a bold step into the unknown, highlighting his unwavering faith in the teachings of Krishna and his commitment to spreading them across the globe. This decision set the stage for the cultural exchange that would soon unfold, marking the beginning of a transformative journey for many.

## Arrival in America

In September 1965, Srila Prabupada arrived in New York City aboard the merchant ship Jaladuta. The bustling metropolis was a stark contrast to the serene landscapes of his homeland. Upon his arrival, he faced immediate challenges, including a lack of familiarity with the culture and the daunting task of finding followers who would resonate with his teachings.

Initially, Prabupada struggled to gain traction in a city filled with distractions. However, his unwavering commitment and authentic approach gradually attracted the attention of seekers of spirituality. It was during this period that he began to form a small community of followers, who resonated with his teachings on devotion, love, and the pursuit of a higher purpose. This grassroots movement laid the foundation for the global presence of ISKCON.

## Establishment of ISKCON

The establishment of ISKCON in 1966 marked a significant milestone in the growth of Krishna consciousness in the West. Under Srila Prabupada’s leadership, the organization flourished, quickly expanding beyond New York to cities across America and eventually the world. Key milestones, such as the opening of the first temple and the celebration of festivals, brought a sense of community and belonging to those seeking spiritual fulfillment.

Prabupada's teachings emphasized the importance of communal living, chanting, and sharing knowledge, which struck a chord with many individuals disillusioned by the materialistic ethos of the time. His ability to adapt ancient Vedic wisdom to contemporary society was a testament to his visionary approach, enabling a diverse audience to connect with and embrace the principles of Krishna consciousness. This adaptability ensured that the message remained relevant across different cultures and eras.

## Legacy and Influence

Srila Prabupada's influence on Western spirituality is profound and enduring. He not only introduced millions to the practices of bhakti yoga but also fostered a broader appreciation for Eastern philosophies. His teachings continue to inspire individuals seeking spiritual depth and community in an increasingly fragmented world.

Furthermore, the ongoing relevance of his work is evident in the rising interest in yoga, meditation, and holistic living today. His legacy is celebrated through numerous temples, festivals, and educational programs worldwide, ensuring that his message of love, devotion, and spiritual awakening continues to resonate. The impact of his teachings invites a reconsideration of spiritual practices and encourages a return to the roots of devotion.

## Conclusion

The journey of Srila Prabupada to America is a compelling narrative of courage, commitment, and transformation. His efforts to spread Krishna consciousness have not only shaped his life but have also significantly influenced the spiritual landscape of the West. As we reflect on his journey, it becomes clear that there is much to learn from his teachings, which emphasize love, selflessness, and the pursuit of truth.

For those seeking deeper spiritual understanding or a sense of community, exploring the teachings of Srila Prabupada and participating in ISKCON activities can be enriching. In a world often overshadowed by material pursuits, his message of devotion and love remains a guiding light, inviting all to embark on their own spiritual journeys. Engaging with his teachings offers a pathway to personal transformation and collective harmony.

## Call to Action

I invite you to explore more about Srila Prabupada’s teachings and ISKCON. Visit your local ISKCON temple, participate in events, or delve into his writings. Experience the beauty of Krishna consciousness and discover how it can enrich your life and foster a sense of connection and purpose. By immersing yourself in these teachings, you may find a renewed sense of direction and fulfillment on your spiritual path.
```

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).